# Stochastic Simulation CW

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import warnings

## Q1a)

#### By analysing the terms inside the unnormalised Banana density, we may choose a relevant proposal q. We have:

$\bar{p}(x_1, x_2) = \exp \left( -\frac{x_1^2}{10} - \frac{x_2^2}{10} - 2(x_2 - x_1^2)^2 \right)$

#### Notice that these terms look similar to terms derived from Gaussian density's. For the conditional density, we also introduce a Gaussian Mixture Model, as it is better suited to covering the original density. We hence use the following:

$q(x_1) = N(x_1; 0, \frac{5}{4})$

$q(x_2|x_1) = \frac{1}{5}\cdot[N(x_2; x_1^{2} - \frac{1}{5}, \frac{1}{5}) + 2 \cdot N(x_2; x_1^{2} - \frac{1}{10}, \frac{2}{5}) + 2 \cdot N(x_2; x_1^{2} - \frac{1}{5}, \frac{1}{5})]$

$q(x_1, x_2) = q(x_1) \cdot q(x_2|x_1) = \frac{1}{\sqrt{2\pi \cdot \frac{5}{4}}} \exp\left(-\frac{x_1^2}{2 \cdot \frac{5}{4}}\right) \cdot \frac{1}{5} \left( 
\frac{1}{\sqrt{2\pi \cdot \frac{1}{5}}} \exp\left(-\frac{\left(x_2 - \left(x_1^2 - \frac{1}{5}\right)\right)^2}{2 \cdot \frac{1}{5}}\right)
+ 2 \cdot \frac{1}{\sqrt{2\pi \cdot \frac{2}{5}}} \exp\left(-\frac{\left(x_2 - \left(x_1^2 - \frac{1}{10}\right)\right)^2}{2 \cdot \frac{2}{5}}\right)
+ 2 \cdot \frac{1}{\sqrt{2\pi \cdot \frac{1}{5}}} \exp\left(-\frac{\left(x_2 - \left(x_1^2 - \frac{1}{5}\right)\right)^2}{2 \cdot \frac{1}{5}}\right)
\right)$

#### We implement the PDF's for both below:

In [2]:
def p_bar(x1, x2):
    return np.exp(-x1**2/10-x2**2/10-2*(x2-x1**2)**2)

def q(x1, x2):
    q_x1 = sp.stats.norm.pdf(x1, loc=0, scale=np.sqrt(5/4))

    # Conditional distributions; mixtures
    q_x2_given_x1_m1 = sp.stats.norm.pdf(x2, loc=x1**2-0.2, scale=np.sqrt(1/5)) 
    q_x2_given_x1_m2 = sp.stats.norm.pdf(x2, loc=x1**2-0.1, scale=np.sqrt(2/5))
    q_x2_given_x1_m3 = sp.stats.norm.pdf(x2, loc=x1**2, scale=np.sqrt(1/5))

    # Weighted GMM; weights = [0.2, 0.4, 0.4]
    q_x2_given_x1 = (q_x2_given_x1_m1 + 2 * q_x2_given_x1_m2 + 2 * q_x2_given_x1_m3) / 5

    return q_x1 * q_x2_given_x1

#### We can show that this is a sensible implementation of a normalised density, by taking the integral over [-12, 12] x [-12, 12], and it being 0.9981 which is very close to 1:

In [3]:
def integrate_q_pdf():
    area, _ = sp.integrate.dblquad(q, -12, 12, -12, 12)
    return area

integrate_q_pdf()

0.9980685670781415

## Q1b)

#### It would be possible to set:

$M = \frac{\bar{p}(x_1, x_2)}{q(x_1, x_2)}$

#### We would then maximise over:

$log(M) = log(\bar{p}(x_1, x_2)) - log(q(x_1, x_2))$

#### However, based on the choice of q, this would be require a relatively involved calculation, so it would be more suitable to search for M using a numerical implementation. We implement a numerical grid search and find that:

$M = \sup_{\substack{(x_1, x_2)}} \frac{\bar{p}(x_1, x_2)}{q(x_1, x_2)} \approx 4.5901 < 4.6$

In [4]:
def find_optimum_ratio():
    # Numerical grid search across a suitable space
    x1_vals = np.linspace(-12, 12, 1000)
    x2_vals = np.linspace(-12, 12, 1000)

    max_ratio = 0

    # Update the max by iterating through all combinations
    for i in range(len(x1_vals)):
        for j in range(len(x2_vals)):
            p_val = p_bar(x1_vals[i], x2_vals[j])
            q_val = q(x1_vals[i], x2_vals[j])

            if(q_val != 0):
                ratio = p_val/q_val
                max_ratio = max(max_ratio, ratio)
            
    return max_ratio

find_optimum_ratio()

# Choose M = 4.6 as it is greater than the numerical supremum

KeyboardInterrupt: 

#### Note that whilst we should be theoretically optimising over R^2, it suffices to optimise over the region of interest which is [-12,12] x [-12,12] by inspection.



## Q1c)

#### In order to simulate the sampling from our proposal density q, we first sample from our marginal density, and then we use a Bernouilli sample to select our index, and then select the appropriate normal distribution from our Gaussian Mixture Model.

In [ ]:
def sample_q_conditional(x1):
    # Bernoulli distribution using uniform distribution to choose the index for the mixture model
    U = np.random.uniform(0,1)

    if 0 <= U and U < 0.2:
        return np.random.normal(x1**2-0.2, np.sqrt(1/5))

    if 0.2 <= U and U < 0.6:
        return np.random.normal(x1**2-0.1, np.sqrt(2/5))

    return np.random.normal(x1**2, np.sqrt(1/5))


def rejection_sampler():
    N = 50000
    M = 4.6

    # Generate x1's by using the marginal density
    x1s = np.random.normal(0, np.sqrt(5/4), N)

    # Store the arrays of accepted entries of x
    x1_accepted = np.array([])
    x2_accepted = np.array([])

    # Iterate through generating x2's using the x1's, and store if the acceptance condition is met
    for i in range(N):
        x2 = sample_q_conditional(x1s[i])
        p_val = p_bar(x1s[i], x2)
        q_val = q(x1s[i], x2)    
        U = np.random.uniform(0,1)

        # Log acceptance condition
        if (np.log(U) < np.log(p_val) - np.log(M) - np.log(q_val)):
            x1_accepted = np.append(x1_accepted, x1s[i])
            x2_accepted = np.append(x2_accepted, x2)

    acceptance_rate = len(x1_accepted) / N
    print(f"Empirical Acceptance Rate: {acceptance_rate:.4f}")

    return x1_accepted, x2_accepted

sample_x1s, sample_x2s = rejection_sampler()
plt.figure(figsize=(8, 8))
plt.scatter(sample_x1s, sample_x2s, s=1, alpha=0.3)
plt.title("Scatter Plot of Accepted Samples")
plt.xlabel("x1")

plt.ylabel("x2")
plt.show()

## Q1d)

In [ ]:
def theoretical_acceptance_rate():
    M = 4.6
    Z, _ = sp.integrate.dblquad(p_bar, -12, 12, -12, 12)
    acceptance_rate = Z / M
    print(f"Theoretical Acceptance Rate: {acceptance_rate:.4f}")

theoretical_acceptance_rate()

## Q1e)

In [ ]:
def kolmogorov_smirnov_test():
    # Load data from samples.npy
    data = np.load('samples.npy')
    banana_x1s = data[:, 0]
    banana_x2s = data[:, 1]

    # Kolomogorov tests
    ks_x1_stat, ks_x1_p_value = sp.stats.ks_2samp(banana_x1s, sample_x1s)
    ks_x2_stat, ks_x2_p_value = sp.stats.ks_2samp(banana_x2s, sample_x2s)

    # Print
    print(f"Kolmogorov-Smirnov Test for x1 marginal: p-value = {ks_x1_p_value}")
    print(f"Kolmogorov-Smirnov Test for x2 marginal: p-value = {ks_x2_p_value}")

kolmogorov_smirnov_test()

## Q2a)

#### Please not that for this question, we will be using a different choice for the proposal density q(x), to simplify calculations:

$q(x_1) = N(x_1; 0, 5)$

$q(x_2|x_1) = N(x_2; x_1^{2}, \frac{1}{4})$

$\implies q(x_1, x_2) = q(x_2|x_1) \cdot q(x_1) = \frac{1}{2 \pi \cdot \sqrt{\frac{5}{4}}} \cdot \exp(-\frac{x_1^{2}}{10} \cdot -2(x_2 - x_1^{2})^{2})$

#### We will derive SNIS estimators for both the marginal distribution as follows:

${p}_1(y_{1:T})_{\substack{\text{SNIS}}} = \frac{\int p_1(y_{1:T}|x) \cdot \bar{p}(x) \, dx}{\int \bar{p}(x) \, dx}$

$\text{Defining } W(x) = \frac{\bar{p}(x)}{q(x)} ,\text{ } \log(\tilde{W}_i) = \log(\bar{p}(X^{(i)})) - \log(q(X^{(i)})) - \max_{\substack{i}} \log(W(X^{(i)})) \text{ and } \bar{w}_i = \frac{\exp(\log(\tilde{W}_i))}{\sum_{i=1}^{N} \exp(\log(\tilde{W}_i))} \text{, we get that:}$

$\int \hat{p}_1(y_{1:T}|x) \cdot \bar{p}(x) \, dx = \int \frac{p_1(y_{1:T}|x) \cdot \bar{p}(x)}{q(x)}\, \cdot q(x) dx = \int p_1(y_{1:T}|x) \cdot W(x) \cdot q(x)\, dx \approx \frac{1}{N} \sum_{i=1}^{N} p_1(y_{1:T}|x^{(i)}) \cdot W(x^{(i)})$

$\int \bar{p}(x)\, dx = \int \frac{\bar{p}(x)}{q(x)} \cdot q(x) \, dx = \int W(x) \cdot q(x) \, dx \approx \frac{1}{N} \sum_{i=1}^{N} W(x^{(i)})$

$\implies \hat{p}_1(y_{1:T})_{\substack{\text{SNIS}}} = \frac{1}{N} \sum_{i=1}^{N} p_1(y_{1:T}|x^{(i)}) \cdot \bar{w}_i$

$\text{We also have that } p_1(y_{1:T}|x^{(j)}) = \prod_{i=1}^{T} N(x_1^{(j)}, \frac{1}{10}) = {(\frac{5}{\pi})}^{\frac{T}{2}} \cdot \exp (-5\sum_{i=1}^{T}{(y_i - x_1^{(j)})}^2)$

$\implies \hat{p}_1(y_{1:T})_{\substack{\text{SNIS}}} = \frac{1}{N} \sum_{i=1}^{N} [{(\frac{5}{\pi})}^{\frac{T}{2}} \cdot \exp (-5\sum_{i=1}^{T}{(y_i - x_1^{(j)})}^2) \cdot \bar{w}_i]$

$\text{Simiarly we have that } p_2^{v}(y_{1:T}|x^{(j)}) = (\frac{\Gamma(\frac{1+v}{2})}{\Gamma(\frac{v}{2}) \cdot (\frac{\pi v}{10})^{\frac{1}{2}}})^{T} \cdot \prod_{i=1}^{T} [1 + \frac{10(y_i - x_j^{(i)})^{2}}{v}]^{-\frac{1+v}{2}} \text{, so by symmetry we get:}$

$\hat{p}_2^{v}(y_{1:T})_{\text{SNIS}} = \frac{1}{N} \sum_{i=1}^{N} p_2^{v}(y_{1:T}|x^{(i)}) \cdot \bar{w}_i = \frac{1}{N} \sum_{i=1}^{N} [\frac{\Gamma(\frac{1+v}{2})}{\Gamma(\frac{v}{2}) \cdot (\frac{\pi v}{10})^{\frac{1}{2}}})^{T} \cdot \prod_{i=1}^{T} [1 + \frac{10(y_i - x_j^{(i)})^{2}}{v}]^{-\frac{1+v}{2}} \cdot \bar{w}_i] $

## Q2b)

#### Below we have the implementations of the naive logsumexp function and the numerically stable logsumexp function. They have been tested below this

In [5]:
def naive_logsumexp(values):
    return np.log(np.sum(np.exp(values)))

def logsumexp(values):
    max_val = np.max(values)
    return np.log(np.sum(np.exp(values - max_val))) + max_val

def test_logsumexp_funcs():
    warnings.simplefilter('error', RuntimeWarning)
    values = np.array([7**i for i in range(10)])

    try:
        res = naive_logsumexp(values)
        print(f"The result of naive_logsumexp is: {res}")
    except RuntimeWarning as e:
        print("Naive implementation failed")

    print(f"The result of numerically stable logsumexp is: {logsumexp(values)}")

test_logsumexp_funcs()

Naive implementation failed
The result of numerically stable logsumexp is: 40353607.0


## Q2c)

In [6]:
# Load the data for y's
ys = np.load('y.npy')

# Redefining the proposal density, q
def q2(x1, x2):
    q2_x1 = sp.stats.norm.pdf(x1, loc=0, scale=np.sqrt(5))
    q2_x2_given_x1 = sp.stats.norm.pdf(x2, loc=x1**2, scale=1/2)
    return q2_x1 * q2_x2_given_x1

#### Implementing SNIS for p1:

In [31]:
# Implementing the non - joint conditional density p(y|x)
def p1(y, x1):
    return sp.stats.norm.pdf(y, loc=x1, scale=np.sqrt(1/10))
    
# Implementing the joint conditional density p(y_1:T|x)
def p1_joint_conditional(ys, x1):
    return np.prod(p1(ys, x1))

def weights(xs):
    x1s = xs[:, 0]
    x2s = xs[:, 1]
    log_Ws = np.log(p_bar(x1s, x2s)) - np.log(q2(x1s, x2s))  
    max_log_w = np.max(log_Ws)
    adjusted_log_Ws = log_Ws - max_log_w
    total = np.sum(np.exp(adjusted_log_Ws))
    return np.exp(adjusted_log_Ws) / total

def p1_SNIS(xs, ys):
    x1s = xs[:, 0]
    x2s = xs[:, 1]
    ws = weights(xs)
    N = len(x1s)
    T = len(ys)

    vals = np.zeros(N, dtype='float64')

    for i in range(N):
        ls = p1(ys, x1s[i])
        ls = ls[ls > 0]
        vals[i] = np.sum(np.log(ls)) + np.log(ws[i])

    log_SNIS = logsumexp(vals) - np.log(N)

    return log_SNIS, vals

In [32]:
xs = np.load('samples.npy')
ys = np.load('y.npy')

e, v = p1_SNIS(xs, ys)

In [16]:
np.log(weights(xs))

array([-10.9659917 , -16.07600761, -11.65965468, ..., -10.7153705 ,
       -10.6976686 , -10.69316527])

In [33]:
e

-18571.133954033347